In [1]:
import tensorflow as tf
import os
import pandas as pd

In [2]:
def make_predictions(test_input_paths, version_nr): 
    
    tf.reset_default_graph()

    export_path_base = "./checkpoint_dir"
    export_path = os.path.join(tf.compat.as_bytes(export_path_base),
                               tf.compat.as_bytes(str(version_nr)))
    
    # IMPORTANT: All keys below need to be the same with the one used when saving
    init_iterator_signature_key = 'init_iterator_key'
    input_key_filenames = 'filenames'
    predict_signature_key = 'predict_classes_key'    
    input_key_training = 'training'
    output_key_input_IDs = 'input_IDs'
    output_key_X_mb = 'X_mb'
    output_key_Y_mb = 'Y_mb'
    output_key_predictions = 'predictions'
    
    prediction_results_file_name = "v" + str(version_nr) + "_prediction_results.csv"
    column_names = ['input_ID', 'prediction result', 'Y (actual labels)']
    raw_data = {column_names[0]: [], 
                column_names[1]: [],
                column_names[2]: []}
    df = pd.DataFrame(raw_data, columns = column_names)
    df.to_csv(prediction_results_file_name, index=False)    
    
    with tf.Session() as sess_predict:
        
        meta_graph_def = tf.saved_model.loader.load(sess_predict,
                                                    [tf.saved_model.tag_constants.SERVING],
                                                    export_path)
        signature = meta_graph_def.signature_def

        # Get tensor namesby using corresponding tensor keys
        filenames_tensor_name = signature[init_iterator_signature_key].inputs[input_key_filenames].name
        training_tensor_name = signature[predict_signature_key].inputs[input_key_training].name
        input_IDs_tensor_name = signature[predict_signature_key].outputs[output_key_input_IDs].name
        X_mb_tensor_name = signature[predict_signature_key].outputs[output_key_X_mb].name
        Y_mb_tensor_name = signature[predict_signature_key].outputs[output_key_Y_mb].name
        predictions_tensor_name = signature[predict_signature_key].outputs[output_key_predictions].name
        
        # Get actual tensors by using tensor names
        filenames_tensor = sess_predict.graph.get_tensor_by_name(filenames_tensor_name)
        training_tensor = sess_predict.graph.get_tensor_by_name(training_tensor_name)
        input_IDs_tensor = sess_predict.graph.get_tensor_by_name(input_IDs_tensor_name)
        X_mb_tensor = sess_predict.graph.get_tensor_by_name(X_mb_tensor_name)
        Y_mb_tensor = sess_predict.graph.get_tensor_by_name(Y_mb_tensor_name)
        predictions_tensor = sess_predict.graph.get_tensor_by_name(predictions_tensor_name)
    
        # Initialize file iterator to read the test data from the csv file
        sess_predict.run(signature[init_iterator_signature_key].method_name, 
                         feed_dict={filenames_tensor: test_input_paths})
        with open(prediction_results_file_name, 'a') as file:
            while True:
                try:
                  # Read the test csv file and make predictions
                  input_IDs, X_mb, Y_mb, predictions = sess_predict.run([input_IDs_tensor, 
                                                                         X_mb_tensor, 
                                                                         Y_mb_tensor,
                                                                         predictions_tensor], 
                                                                         feed_dict={training_tensor: False})

                  #print("input_IDs: \n", input_IDs)
                  #print("X_mb: \n", X_mb)
                  #print("Y_mb: \n", Y_mb)
                  #print("predictions: \n", predictions)
                  raw_data = {column_names[0]: input_IDs, 
                              column_names[1]: predictions,
                              column_names[2]: Y_mb.tolist()}
                  df = pd.DataFrame(raw_data, columns = column_names)
                  df.to_csv(file, header=False, index=False)
                except tf.errors.OutOfRangeError:
                  print("All examples in the test file(s) have been processed")
                  break         


In [3]:
test_path1 = "test1.csv"
test_input_paths = [test_path1]

version_nr = 80

make_predictions(test_input_paths, version_nr)

INFO:tensorflow:Restoring parameters from b'./checkpoint_dir/80/variables/variables'
All examples in the test file(s) have been processed


In [ ]:
import numpy as np
aa = np.array([[1, 2, 3], [4, 5,6]])

print(aa.shape)
print(aa[:,-1])
print(aa[:,0])

In [ ]:
bb = np.array([[0], [0], [1]])
print(bb.shape)